# ________________Musk Notifier__________________

## Introduction and problem statement
Wiki : Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is an engineer, industrial designer, and technology entrepreneur. In December 2016, he was ranked 21st on the Forbes list of The World's Most Powerful People, and was ranked joint-first on the Forbes list of the Most Innovative Leaders of 2019.

Videos of Elon Musk are quite popular on YouTube, with a lot of people watching from various walks of life. Painting the picture of a genius entreprenuer he imbibes the role of a celebrity figure. His videos are often watched for inspiration, as food for thought and sometimes only for his supposed eccentricity. This has led to Elon having a lot of followers, including Youtubers that hold discussions about him on their own channels. Due to this and his name being used abundantly in plenty of content online, one might find it hard to filter and catch hold of new videos that actually have him in it as opposed to having some other Youtubers discussing him. 

## Solution proposal
The aim is to seach YouTube daily for videos with Elon in them and save the links to those videos in a file. Those videos are to be checked that appear in results when searched for 'Elon Musk'. The results are filtered to show only those videos which have been published in the last 24 hours (Since the script is meant once every day).

A face detection package is used and in order to recognise the face as Elon's, a machine learning model is made use of. The model is trained using faces of Elon. Still images from the videos to be checked for Elon are captured. The faces are isolated and a prediction is made using the trained model. If the number Elon Musk faces in the (multiple) images of a video are predicted to be more than a predefined threshold, then that video is flagged to have Elon in it. Finally the list of links are saved in a file on the desktop.

### Other possible solutions
(.) The face detection package used can also recognise faces, but instead I chose to build a machine learning model myself.
(.) Instead of only taking the still images from the videos, the whole video could be made use of for classification into categories: With Elon and Without Elon. But that seems to be bandwidth heavy and alon harder (read longer) to train. I think the solution can be less hungry for data.
(.) Audio could be made use of in training instead. But again the entire audio length will have to be made use of.
(.) One might think that the title of the video or it's description or channel name can be used to tell if the video contains Elon. But since videos with Elon are released by multiple sources, the channels can't be kept track of. Some of the videos do not even have a description, hence using the video metadata for classification might not turn out to be such a great idea.

## Methodology
### Elon Musk videos are searched for on YouTube
 Selenium is used to go to each video link. The links are to be manually collected initially, for training purposes. Selenium then takes screenshots in regular intervals. It achieves this by seeking through the video. Once a particular video is done, it moves on to the next video and repeats until all videos have had their still images collected. 
### Faces are extracted and used for training
 As soon as there is a screenshot taken, all the faces in the image are extracted using the 'face_recognition' package and saved in the 'save_path' location provided. The faces are then manually sorted into two classes. One that is Elon's face and another which is not Elon's face. These images are then used to train a CNN model. The trained model is then saved and used to classify new videos.
### New videos are checked for Elon
 'Elon Musk' is searched for on YouTube. The results are filtered to show only videos from the past 24 hours and then the first 10 videos videos have their still images taken at regular intervals. If a video has more than a predefined percentage of images that test positive for Elon, then that link is flagged and appended to be saved in a file after all the other videos have been checked.

In [40]:
# Takes YouTube links or a search term as parameter and saves list of videos with Elon. 
# Either 'links' or 'search_term' must be provided for it to work. If 'delete_images' is
# set to true then the images are deleted after analysis.
def get_faces(links = None, save_path , search_term = None, delete_images = False): 
    
    #Selenium to save the still images of each video.
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver import ActionChains
    
    #To load the trained model for prediction and for image preprocessing.
    from tensorflow.keras.preprocessing import image as tf_image
    from tensorflow.keras.models import load_model
    
    #For recognising faces, saving and navigating directories.
    import matplotlib.pyplot as plt
    import face_recognition
    import os
    import shutil
    import time
    import csv
    import numpy as np
    
    image_classifier = load_model(r"C:\Users\Anurag\Desktop\biggySmalls")
    
    # Initialising filename for each face image, full image(debugging purposes)
    # and the list to append links with elon.
    image_save_number = 1
    main_image_save_number = 1   
    links_with_elon = [] 
    
    #Takes the first ten links in the results page for the given 'search_term'.   
    if search_term:
        
        options = Options()
        path_to_extension = r"C:\Users\Anurag\1First\Projects\EM files\3.7_0"
        options.add_argument("--headless")
        options.add_argument("load-extension=" + path_to_extension)
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")
        options.add_argument("--mute-audio")
        driver=webdriver.Chrome(options=options, executable_path=r"C:\Users\Anurag\Downloads\chromedriver_win32\chromedriver.exe")
        
        driver.get("https://www.youtube.com/")
        time.sleep(3)
        WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#search"))).send_keys(search_term)
        driver.find_element_by_css_selector("button.style-scope.ytd-searchbox#search-icon-legacy").click()
        time.sleep(3)
        #Filter results to show only the videos uploaded in the last 24 hours.
        driver.find_element_by_xpath("/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/paper-button").click()
        driver.find_element_by_xpath("/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[2]/a/div").click()
        time.sleep(5)
        #Find and save the first ten links.
        links = [my_href.get_attribute("href") for my_href in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-video-renderer#video-title")))][:10]
        driver.quit()
        
    #Loading adblocker extension on to Chrome.
    chrome_options = Options()
    path_to_extension = r"C:\Users\Anurag\1First\Projects\EM files\3.7_0"
    chrome_options.add_argument("load-extension=" + path_to_extension)
    #chrome_options.add_argument("--headless")
    #Since headless chrome can't go fullscreen, photo quality is affected. Do not use.
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--window-size=1200,1100")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--mute-audio")
    #Instance variable for Chrome browser.
    driver = webdriver.Chrome(r"C:\Users\Anurag\Downloads\chromedriver_win32\chromedriver.exe",options=chrome_options)
            
    def visit_links_and_get_faces(link):
        nonlocal image_save_number
        nonlocal main_image_save_number
        nonlocal links_with_elon 
        
        def seek(url_time):
            #Seeking by appending time to the URL of the video.
            url_with_time = "https://youtu.be/" + (link.split("watch?v="))[1] + "?t=" + str(url_time)
            driver.get(url_with_time)
            time.sleep(2)
            #To fullscreen and get a higher quality image.
            driver.find_element_by_css_selector("#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-right-controls > button.ytp-fullscreen-button.ytp-button").click()
#             element = driver.find_element_by_css_selector("#movie_player > div.html5-video-container > video")
#             driver.execute_script("arguments[0].click();", element)
            
        def get_video_screeshot_timings():
            # Get the length of the video in order to determine 
            # how many images need to be collected.
            video_length = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"""#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-left-controls > div > span.ytp-time-duration"""))).text
            hh_mm_ss = [int(i) for i in video_length.split(':')]
            # If the video is less than an hour long, append dummy zeros
            # in order to have uniform length for "hh_mm_ss".
            if len(hh_mm_ss) < 3:    
                hh_mm_ss.insert(0,0) 
            length_in_seconds = (hh_mm_ss[0]*3600) + (hh_mm_ss[1]*60) + (hh_mm_ss[2])
            no_of_screenshots =  length_in_seconds//60 #A screenshot every minute (60s)

            if no_of_screenshots == 0: #In case of short videos (<=2mins) take. 
                no_of_screenshots = 4  # only 4 screenshots per video.
            elif no_of_screenshots > 30: #Take 30 screenshots for long videos.
                no_of_screenshots = 30
            #'url_time_period' is the interval between each screenshot for a particular video.    
            url_time_period = length_in_seconds//no_of_screenshots 
            return length_in_seconds, url_time_period

        #Create the 'save_path' directory.
        os.makedirs(save_path, exist_ok = True)

        #Go to YouTube page with video.
        driver.get(link) 
        
        #Get time interval for this video between every screenshot.
        length_in_seconds, url_time_period = get_video_screeshot_timings()
        
        #Take a screenshot once every 'url_time_period' interval and save the faces.  
        for url_time in range(1, length_in_seconds, url_time_period):
            seek(url_time)
            image_address = save_path + "\\temporary.png"
            driver.save_screenshot(image_address)   
            
            #Load the image onto the face recognition package and find coordinates of faces.
            image = face_recognition.load_image_file(image_address)
            face_locations = face_recognition.face_locations(image)
             
            #"link[-5:]" in "face_save_path" used to group all photos from same video later.
            face_save_path = save_path + "\\" + link[-5:] + "_" + str(image_save_number) + ".png"
            #The main image is saved only for debugging purposes.
            image_save_path = save_path + "\\" + str(main_image_save_number) + "OGG" + ".png"
            main_image_save_number += 1
            plt.imsave(image_save_path, image)
            #One coordinate for each corner of the face square.
            for coordinates in face_locations:
                plt.imsave(face_save_path, image[coordinates[0]:coordinates[2],coordinates[3]:coordinates[1]])
                image_save_number += 1
                
            os.remove(image_address)
        #-----------------------------PREDICTION------------------------#
        # Prediction is only made for the videos when 'search_term' is provided. 
        # If 'search_term' is not provided, but 'links' is, then it means that 
        # images need to be downloaded for training and there is no reason to 
        # make prediction in that case
        if search_term:
            images_by_video = []
            
            # Going through all the images for current video
            # in 'save_path' and  preprocessing them.
            for root, dirs, files in os.walk(save_path):
                for name in files:
                    if name[:5] in link:
                        img = tf_image.load_img(os.path.join(root,name), target_size = (64, 64))
                        img = tf_image.img_to_array(img)
                        img = np.expand_dims(img, axis = 0)
                        images_by_video.append(img)

            results = []   #To save the predictions for all images from current video.
            
            #Making predictions and appending the result.
            for single_image in images_by_video:
                result = (image_classifier.predict(single_image))[0][0]
                results.append(result)
    
            if len(results) == 0:  #Go to next link if no faces are detected.
                return

            # If Elon is present the result is 0 else it is 1. So if there are more than 
            # 60% of images that test true for Elon then append the current to the final list.
            elon_q = sum(results)/len(results)
            threshold = 0.4  #0.4 is the complement of 60%.

            if elon_q < threshold:
                links_with_elon.append(link)
            
        return
    
    for link in links:
        try:
            # Sometimes live videos are parsed and this causes an error. 
            # Some other times, due to Selenium there can be 'timeout' errors.
            # Overall errors are limited to only those two situations.
            visit_links_and_get_faces(link)
        except Exception as e: print(e)
    
    if delete_images:
        shutil.rmtree(save_path)
    # Attempt to save the list only when 'search_term' is provided and
    # not when 'links' is provided.
    if search_term:
        locationss = r"C:\Users\Anurag\Desktop\linksWithElon.csv"
        with open(locationss,"w") as f:
            wr = csv.writer(f,delimiter="\n")
            wr.writerow(links_with_elon)
        
    return

In [41]:
#Runs 24 hours after the last time it terminated.
import time
while True:
    get_faces(save_path=r"C:\Users\Anurag\Desktop\Oorsule",search_term="elon musk")
    time.sleep(60*60*24)

['https://www.youtube.com/watch?v=5rEaoOvTNbY', 'https://www.youtube.com/watch?v=T0HODkk1FxA', 'https://www.youtube.com/watch?v=hMzWRwD8GxQ', 'https://www.youtube.com/watch?v=BqCnVkOXsBc', 'https://www.youtube.com/watch?v=j-zlgcgGG40', 'https://www.youtube.com/watch?v=A3nXYtU0hMs', 'https://www.youtube.com/watch?v=1ioyqmzHii4', 'https://www.youtube.com/watch?v=JEiCMVSoqpA', 'https://www.youtube.com/watch?v=BKk2EXYOHAk', 'https://www.youtube.com/watch?v=MD6-fizoTeI']
https://youtu.be/5rEaoOvTNbY?t=1
https://youtu.be/5rEaoOvTNbY?t=67
https://youtu.be/5rEaoOvTNbY?t=133
vTNbY_1.png 1.0
vTNbY_2.png 1.0
vTNbY_4.png 1.0
[1.0, 1.0, 1.0]
https://youtu.be/T0HODkk1FxA?t=1
https://youtu.be/T0HODkk1FxA?t=64
https://youtu.be/T0HODkk1FxA?t=127
https://youtu.be/T0HODkk1FxA?t=190
https://youtu.be/T0HODkk1FxA?t=253
https://youtu.be/T0HODkk1FxA?t=316
https://youtu.be/T0HODkk1FxA?t=379
https://youtu.be/T0HODkk1FxA?t=442
https://youtu.be/T0HODkk1FxA?t=505
https://youtu.be/T0HODkk1FxA?t=568
https://youtu.be

## Training a model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

test_path = r"C:\Users\Anurag\1First\Projects\EM files\Seperate_faces\To feed tensorflow\Test"
train_path = r"C:\Users\Anurag\1First\Projects\EM files\Seperate_faces\To feed tensorflow\Train"
training_set = train_datagen.flow_from_directory(train_path,target_size = (64, 64),batch_size = 32,class_mode = 'binary')
test_set = test_datagen.flow_from_directory(test_path,target_size = (64, 64),batch_size = 32,class_mode = 'binary')

In [ ]:
class_weight = {0: 1,
                1: 3.5}
#Make it 100 epoch and class_weight 1:3.5 for result  melon 3:23, nelon 8:42
classifier.fit_generator(training_set,
steps_per_epoch = 8000,
epochs = 100,
validation_data = test_set,
class_weight = class_weight,
validation_steps = 2000)

In [ ]:
training_set.class_indices

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os
counter_right = 0
counter_wrong = 0
for root, dirs, files in os.walk(r"C:\Users\Anurag\1First\Projects\EM files\Seperate_faces\To feed tensorflow\Val\Melon"):
    for name in files:
        image_address = os.path.join(root,name)
        test_image = image.load_img(image_address, target_size = (64, 64))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        print(result, name)
        if result[0][0] == 1:
            counter_right += 1
        else:
            counter_wrong += 1
print(counter_right, counter_wrong)

In [ ]:
classifier.save(r"C:\Users\Anurag\Desktop\biggySmalls")